## 1. Import Data

In [1]:
import pickle
import gzip
import numpy as np
import os
import cv2

In [2]:
def load_zipped_pickle(filename):
    with gzip.open(filename, 'rb') as f:
        loaded_object = pickle.load(f)
        return loaded_object

In [3]:
def save_zipped_pickle(obj, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(obj, f, 2)

In [4]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
        assert p['name'] == t['name']
        prediction = np.array(p['prediction'], dtype=bool)
        target = np.array(t['label'], dtype=bool)

        assert target.shape == prediction.shape
        overlap = prediction*target
        union = prediction + target

        ious.append(overlap.sum()/float(union.sum()))
    
    print("Median IOU: ", np.median(ious))

## 1.1 Load Data

In [5]:
# load data
train_data = load_zipped_pickle("train.pkl")
test_data = load_zipped_pickle("test.pkl")
samples = load_zipped_pickle("sample.pkl")



数据结构：  
    train_data[1-65]包含65个病人的数据, 前46个是amateur数据， 后19个是expert数据  
    train_data[1]['video','label','dataset'...] , video是图片数组， label是MV mask，dataset代表是amateur还是expert  
    train_data[1]['video'] shape: [IMG_WIDTH, IMG_HEIGHT, FRAME NUM]， label也一样

In [6]:
# test_img = train_data[60]['label'][:,:,24]
# zero_img = np.zeros(test_img.shape)
# zero_img[test_img] = 255
# zero_img_resize = cv2.resize(zero_img, (1000,1000))
# zero_img.shape

In [7]:
# cv2.imshow("mask", zero_img_resize)
  
# cv2.waitKey(0) 

# cv2.destroyAllWindows() 

## 1.2 Data pre-processing

In [6]:
# 样本图片大小
IMG_WIDTH = 256
IMG_HEIGHT = 256
IMG_CHANNELS = 1

In [7]:
# 找到含有标记的帧
def is_labeled(mask_list):
    
    label_index = []
    for i in range((mask_list).shape[2]):
        if(True in mask_list[:,:,i]):
            label_index.append(i)
            continue
    return np.array(label_index)

def image_size_normalize(raw_data, size):
    #选出有标记的图片并resize到相应尺寸
    img_width, img_height, img_channel = size
    
    labeled_img_list = []
    mask_list = []
    
    for patient_i in range(len(raw_data)):

        img_i = raw_data[patient_i]['video'] # Shape: [IMG_WIDTH1, IMG_HEIGHT1, Frame_Num1]
        mask_i = raw_data[patient_i]['label']
        label_index = is_labeled(mask_i)
        labeled_img_i = img_i[:,:,label_index]
        labeled_mask_i = mask_i[:,:,label_index]
        
        for frame_j in range(labeled_mask_i.shape[2]):
            img_j = labeled_img_i[:,:,frame_j] # [IMG_WIDTH2, IMG_HEIGHT2, Frame_Num2]
            mask_j = labeled_mask_i[:,:,frame_j]
            mask_j_digi = np.zeros(mask_j.shape)
            mask_j_digi[mask_j] = 1
            if(img_j.shape[0]!= IMG_WIDTH or img_j.shape[1]!= IMG_HEIGHT):
                img_j = cv2.resize(img_j,(img_width, img_height))
                mask_j_digi = cv2.resize(mask_j_digi,(img_width, img_height))
            labeled_img_list.append(img_j)
            mask_list.append(mask_j_digi)
            
    return np.array(labeled_img_list).reshape(-1,img_width, img_height, img_channel),np.array(mask_list).reshape(-1,img_width, img_height, 1)

In [9]:
# 从训练集中选出所有label过的图像并resize到指定大小
img_labeled, mask = image_size_normalize(train_data, (IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))  # OutPut Shape: [Labeled_Frame_Num, IMG_WIDTH2, IMG_HEIGHT2, IMG_CHANNEL]

## 2 U-Net

## 2.1Model Definition 

In [10]:
# for i in range(len(train_data)):
#     print(train_data[i]['dataset'])

In [13]:
# for i in range(len(train_data)):
#     print(len(is_labeled(train_data[i]['label'])))


In [11]:
import tensorflow as tf
from tensorflow import keras
import os
import numpy as np
from tqdm import tqdm
# from skimage.io import imread, imshow
# from skimage.transform import resize
import matplotlib.pyplot as plt
import random

In [12]:
inputs = tf.keras.layers.Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)

In [13]:
#Contraction path
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(s)
c1 = tf.keras.layers.Dropout(0.1)(c1)
c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
c2 = tf.keras.layers.Dropout(0.1)(c2)
c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)
 
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
c3 = tf.keras.layers.Dropout(0.2)(c3)
c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)
 
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
c4 = tf.keras.layers.Dropout(0.2)(c4)
c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)
 
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
c5 = tf.keras.layers.Dropout(0.3)(c5)
c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#Expansive path 
u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
u6 = tf.keras.layers.concatenate([u6, c4])
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
c6 = tf.keras.layers.Dropout(0.2)(c6)
c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)
 
u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
u7 = tf.keras.layers.concatenate([u7, c3])
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
c7 = tf.keras.layers.Dropout(0.2)(c7)
c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)
 
u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
u8 = tf.keras.layers.concatenate([u8, c2])
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
c8 = tf.keras.layers.Dropout(0.1)(c8)
c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)
 
u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
c9 = tf.keras.layers.Dropout(0.1)(c9)
c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)
 
outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)
 
model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

2021-12-19 21:21:52.385151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-19 21:21:52.510805: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-19 21:21:52.511597: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:925] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2021-12-19 21:21:52.513667: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 lambda (Lambda)                (None, 256, 256, 1)  0           ['input_1[0][0]']                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 16  160         ['lambda[0][0]']                 
                                )                                                                 
                                                                                              

## 2.2 Train

In [14]:
results = model.fit(img_labeled, mask, batch_size=10, epochs=10)

Epoch 1/10


2021-12-19 21:22:03.708390: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2021-12-19 21:22:05.546980: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.


20/20 [==============================] - 12s 191ms/step - loss: 0.2398 - accuracy: 0.9489
Epoch 2/10
20/20 [==============================] - 2s 94ms/step - loss: 0.0474 - accuracy: 0.9883
Epoch 3/10
20/20 [==============================] - 2s 95ms/step - loss: 0.0313 - accuracy: 0.9883
Epoch 4/10
20/20 [==============================] - 2s 94ms/step - loss: 0.0257 - accuracy: 0.9883
Epoch 5/10
20/20 [==============================] - 2s 95ms/step - loss: 0.0210 - accuracy: 0.9883
Epoch 6/10
20/20 [==============================] - 2s 95ms/step - loss: 0.0194 - accuracy: 0.9883
Epoch 7/10
20/20 [==============================] - 2s 95ms/step - loss: 0.0177 - accuracy: 0.9883
Epoch 8/10
20/20 [==============================] - 2s 93ms/step - loss: 0.0162 - accuracy: 0.9885
Epoch 9/10
20/20 [==============================] - 2s 90ms/step - loss: 0.0155 - accuracy: 0.9892
Epoch 10/10
20/20 [==============================] - 2s 95ms/step - loss: 0.0155 - accuracy: 0.9902


In [18]:
img_labeled

array([[[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[2],
         [2],
         [1],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]]],


       [[[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        [[0],
         [0],
         [0],
         ...,
         [0],
         [0],
         [0]],

        ...,

        [[1],
         [1],
         [0],
         ...,
         [0],


In [16]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
#         assert p['name'] == t['name']
#         prediction = np.array(p['prediction'], dtype=bool)
#         target = np.array(t['label'], dtype=bool)

        assert targets.shape == predictions.shape
        overlap = predictions*targets
        union = predictions + targets

        ious.append(overlap.sum()/float(union.sum()))
    
#     print("Median IOU: ", np.median(ious))
    return(np.median(ious))
# evaluate(y_pred,y_test)

## cross validation

In [17]:
from sklearn.model_selection import KFold
import sklearn
scores = []
kf = KFold(n_splits=5, shuffle=True)
x_select = img_labeled
y_select = mask
for train_idx, test_idx in kf.split(x_select):
    x_train = x_select[train_idx]
    y_train = y_select[train_idx]
    x_test = x_select[test_idx]
    y_test = y_select[test_idx]
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
#     s = r2_score(y_test.ravel(), y_pred.ravel())
    s= evaluate(y_pred,y_test)
    print('score: ', s)
    scores.append(s)

scores

5/5 [==============================] - 2s 270ms/step - loss: 0.0143 - accuracy: 0.9910
score:  0.20974456774782418
5/5 [==============================] - 1s 271ms/step - loss: 0.0134 - accuracy: 0.9914
score:  0.24150955420454173
5/5 [==============================] - 1s 273ms/step - loss: 0.0133 - accuracy: 0.9914
score:  0.2213668092451906
5/5 [==============================] - 1s 272ms/step - loss: 0.0134 - accuracy: 0.9913
score:  0.23095934400712137
5/5 [==============================] - 1s 275ms/step - loss: 0.0133 - accuracy: 0.9915
score:  0.2634734392851498


[0.20974456774782418,
 0.24150955420454173,
 0.2213668092451906,
 0.23095934400712137,
 0.2634734392851498]

In [48]:
def jaccard_similarity(x,y):
    intersection_cardinality = len(set.intersection(*[set(x), set(y)]))
    union_cardinality = len(set.union(*[set(x), set(y)]))
    return intersection_cardinality/float(union_cardinality)

print(jaccard_similarity(y_test.ravel(),y_pred.ravel()))

1.363456015590665e-06


In [51]:
def correlation(a,b):
    set_a = set(a)
    set_b = set(b)
    unions = len(set_a.union(set_b))
    intersections = len(set_a.intersection(set_b))
    return 1. * intersections / unions
correlation(y_test.ravel(),y_pred.ravel())

1.363456015590665e-06

In [61]:
def evaluate(predictions, targets):
    ious = []
    for p, t in zip(predictions, targets):
#         assert p['name'] == t['name']
#         prediction = np.array(p['prediction'], dtype=bool)
#         target = np.array(t['label'], dtype=bool)

        assert targets.shape == predictions.shape
        overlap = predictions*targets
        union = predictions + targets

        ious.append(overlap.sum()/float(union.sum()))
    
#     print("Median IOU: ", np.median(ious))
    return(np.median(ious))
# evaluate(y_pred,y_test)

In [36]:
import pandas as pd
pd.DataFrame(y_test.flatten()).to_csv("C:/Users/Junling_W/Desktop/y_test",index = False)

In [39]:

pd.DataFrame(mask.flatten()).to_csv("C:/Users/Junling_W/Desktop/mask.csv",index = False)

In [37]:

pd.DataFrame(y_pred.flatten()).to_csv("C:/Users/Junling_W/Desktop/y_pred",index = False)

In [30]:
y_pred

array([[[[3.10724545e-02],
         [2.87532527e-03],
         [1.71948894e-04],
         ...,
         [1.05960009e-06],
         [3.94177696e-05],
         [6.05358393e-04]],

        [[7.92971067e-03],
         [2.14057698e-04],
         [1.89728257e-06],
         ...,
         [5.40964495e-10],
         [8.54724149e-08],
         [4.10475441e-05]],

        [[2.16715271e-03],
         [5.16138152e-06],
         [2.67924332e-08],
         ...,
         [5.60880444e-13],
         [4.97218844e-09],
         [2.13269658e-07]],

        ...,

        [[3.13859346e-04],
         [2.30226897e-06],
         [2.50392080e-08],
         ...,
         [1.24494581e-10],
         [2.10939373e-08],
         [5.77700303e-05]],

        [[5.12614800e-03],
         [1.37194686e-04],
         [1.80893471e-06],
         ...,
         [2.76161991e-08],
         [1.36795870e-06],
         [2.34772029e-04]],

        [[1.19859129e-02],
         [2.03054908e-04],
         [7.49339306e-05],
         ...,
 

In [41]:
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
s = r2_score(y_test.ravel(), y_pred.ravel())

In [42]:
s

0.5186819779734481

## 2.3 MODEL PREDICT

In [16]:
# 从训练集中选出所有label过的图像并resize到指定大小
img_labeled= image_size_normalize(test_data, (IMG_WIDTH,IMG_HEIGHT,IMG_CHANNELS))  # OutPut Shape: [Labeled_Frame_Num, IMG_WIDTH2, IMG_HEIGHT2, IMG_CHANNEL]

KeyError: 'label'

In [ ]:
model.fit(img_labeled, mask, batch_size=10, epochs=10)

# 3 Visualization

In [18]:
import os

from PIL import Image

def video_make(name, image_list, mask_list,  img_size =(112, 112)):

    fps = 24 #视频每秒24帧
    size = img_size #需要转为视频的图片的尺寸

    #视频保存在当前目录下

    fourcc = cv2.VideoWriter_fourcc(*'MJPG')    
    video = cv2.VideoWriter(name+".avi",fourcc, fps,size, False)
    #draw stuff that goes on every frame here
    for frame_i in range(image_list.shape[2]):
        img = (image_list[:,:,frame_i])
        mask = mask_list[:,:,frame_i]
        img[mask] = 255
        img_mat = cv2.Mat(img)
        video.write(img)
    video.release()


## 4 Conclusion（things to be done for improvement）  
4.1 如上可见，随着epoch增加, accuracy波动很大，网络不是很work。 不清楚是数据量不够，网络复杂度不够， 还是iter比较小（纯ml小白）  
    如果是数据量不够， 或许需要用tracking的办法增加数据量
4.2 需要做cross-validation的预测版本
4.3 由于数据集中， 属于MV的像素点很少，和类别不平衡问题很像， 最好能增大 MV区域误识别的错误权重
4.4 预测的可视化实现